In [1]:
import nltk
import random
from subprocess import call
import os
os.environ['PATH'] += os.pathsep + '/usr/local/bin'

In [2]:
sentence = "A man and a woman standing in the kitchen"

In [3]:
tokens = nltk.word_tokenize(sentence)
tokens

['A', 'man', 'and', 'a', 'woman', 'standing', 'in', 'the', 'kitchen']

In [4]:
tagged = nltk.pos_tag(tokens)
tagged

[('A', 'DT'),
 ('man', 'NN'),
 ('and', 'CC'),
 ('a', 'DT'),
 ('woman', 'NN'),
 ('standing', 'VBG'),
 ('in', 'IN'),
 ('the', 'DT'),
 ('kitchen', 'NN')]

In [5]:
entities = nltk.chunk.ne_chunk(tagged)

In [6]:
insults = ["ugly", "fat", "smelly","greasy","dirty","stupid as fuck","lame"]

In [92]:
transformed = []
for currentpair in tagged:
    if (currentpair[1]=='NN'): transformed.append(random.choice(insults))
    transformed.append(currentpair[0])
call(["say",str(transformed)])


0

In [90]:
print transformed

['A', 'stupid as fuck', 'man', 'and', 'a', 'ugly', 'woman', 'standing', 'in', 'the', 'smelly', 'kitchen']


In [133]:
# Used when tokenizing words
sentence_re = r'''(?x)      # set flag to allow verbose regexps
      ([A-Z])(\.[A-Z])+\.?  # abbreviations, e.g. U.S.A.
    | \w+(-\w+)*            # words with optional internal hyphens
    | \$?\d+(\.\d+)?%?      # currency and percentages, e.g. $12.40, 82%
    | \.\.\.                # ellipsis
    | [][.,;"'?():-_`]      # these are separate tokens
'''

lemmatizer = nltk.WordNetLemmatizer()
stemmer = nltk.stem.porter.PorterStemmer()

#Taken from Su Nam Kim Paper...
grammar = r"""
    NBAR:
        {<NN.*|JJ>*<NN.*>}  # Nouns and Adjectives, terminated with Nouns
        
    NP:
        {<NBAR>}
        {<NBAR><IN><NBAR>}  # Above, connected with in/of/etc...
"""
chunker = nltk.RegexpParser(grammar)

toks = nltk.regexp_tokenize(sentence, sentence_re)
postoks = nltk.tag.pos_tag(tokens)
tree = chunker.parse(postoks)

from nltk.corpus import stopwords
stopwords = stopwords.words('english')


In [134]:
def leaves(tree):
    """Finds NP (nounphrase) leaf nodes of a chunk tree."""
    for subtree in tree.subtrees(filter = lambda t: t.label()=='NP'):
        yield subtree.leaves()

In [135]:
def normalise(word):
    """Normalises words to lowercase and stems and lemmatizes it."""
    word = word.lower()
    word = stemmer.stem(word)
    word = lemmatizer.lemmatize(word)
    return word

In [136]:
def acceptable_word(word):
    """Checks conditions for acceptable word: length, stopword."""
    accepted = bool(2 <= len(word) <= 40
        and word.lower() not in stopwords)
    return accepted


In [137]:
def get_terms(tree):
    for leaf in leaves(tree):
        term = [ normalise(w) for w,t in leaf if acceptable_word(w) ]
        yield term
        

In [138]:
terms = get_terms(tree)

In [143]:
for term in terms:
    for word in term:
        print word,
    print

In [229]:
# analyse every term in a sentence and extract persons and objects:

from nltk.corpus import wordnet as wn

wn_entity = wn.synset('entity.n.01')
wn_physical_entity = wn.synset('physical_entity.n.01')
wn_causal_agent = wn.synset('causal_agent.n.01')
wn_person = wn.synset('person.n.01')
wn_object = wn.synset('object.n.01')
# person hierarchy: entity->ph_entity->cas_agent->person
# object hierarchy: entity->ph_entity->object
persons = []
objects = []
maxdepth = 4
for term in get_terms(tree):
    for word in term:
        synset = wn.synsets(word)[0].hypernym_paths()[0]
        #print word, ":"
        #print synset,
        if (wn_object in synset[:maxdepth]): objects.append(word)
        if (wn_person in synset[:maxdepth]): persons.append(word)

print "objects:",objects
print "persons:",persons

objects: ['kitchen']
persons: ['man', 'woman']


In [198]:
print wn.synsets('woman')[0].res_similarity(wn.synset('man.n.01'))
tree

TypeError: res_similarity() takes at least 3 arguments (2 given)

In [55]:
rd_parser = nltk.RecursiveDescentParser(grammar)

In [62]:
for tr in rd_parser.parse(entities):
    print(tr)

AttributeError: 'str' object has no attribute 'check_coverage'

In [54]:
from spacy.en import English
parser = English()
#usnt = unicode('I shot an elephant')
usnt = unicode(sentence)
parsedData=parser(usnt)
#nsubjpass
sub_toks = [tok for tok in parsedData if (tok.dep_ == "nsubj") ]
print sub_toks
for tok in parsedData:   
        print(tok, tok.dep_)

[]
(A, u'det')
(man, u'ROOT')
(and, u'cc')
(a, u'det')
(woman, u'conj')
(standing, u'acl')
(in, u'prep')
(the, u'det')
(kitchen, u'pobj')


In [3]:
example = unicode("The boy with the spotted dog quickly ran after the firetruck.")
parsedEx = parser(unicode(sentence))
for token in parsedEx:
    print(token.orth_, token.dep_, token.head.orth_, [t.orth_ for t in token.lefts], [t.orth_ for t in token.rights])

NameError: name 'sentence' is not defined

In [59]:
# Let's see if it can figure out this analogy
# Man is to King as Woman is to ??
king = parser.vocab[unicode('king')]
man = parser.vocab[unicode('man')]
woman = parser.vocab[unicode('woman')]

result = king.vector - man.vector + woman.vector

# gather all known words, take only the lowercased versions
allWords = list({w for w in parser.vocab if w.has_vector and w.orth_.islower() and w.lower_ != "king" and w.lower_ != "man" and w.lower_ != "woman"})
# sort by similarity to the result
allWords.sort(key=lambda w: cosine(w.vector, result))
allWords.reverse()
print("\n----------------------------\nTop 3 closest results for king - man + woman:")
for word in allWords[:3]:   
    print(word.orth_)
    
# it got it! Queen!


----------------------------
Top 3 closest results for king - man + woman:
queen
prince
kings


In [4]:
from numpy import dot
from numpy.linalg import norm

# you can access known words from the parser's vocabulary
nasa = parser.vocab[unicode('ugly')]

# cosine similarity
cosine = lambda v1, v2: dot(v1, v2) / (norm(v1) * norm(v2))

# gather all known words, take only the lowercased versions
allWords = list({w for w in parser.vocab if w.has_vector and w.orth_.islower() and w.lower_ != "nasa"})

# sort by similarity to NASA
allWords.sort(key=lambda w: cosine(w.vector, nasa.vector))
allWords.reverse()
print("Top 10 most similar words to NASA:")
for word in allWords[:10]:   
    print(word.orth_)

Top 10 most similar words to NASA:
ugly
hideous
awful
fugly
horrible
disgusting
stupid
unattractive
nasty
horrid


In [17]:
# Let's see if it can figure out this analogy
# Man is to King as Woman is to ??
king = parser.vocab[unicode('ugly')]
man = parser.vocab[unicode('man')]
woman = parser.vocab[unicode('woman')]

#result = king.vector*man.vector
result = woman.vector+man.vector

# gather all known words, take only the lowercased versions
allWords = list({w for w in parser.vocab if w.has_vector and w.orth_.islower() and w.lower_ != "king" and w.lower_ != "man" and w.lower_ != "woman"})
# sort by similarity to the result
allWords.sort(key=lambda w: cosine(w.vector, result))
allWords.reverse()
print("\n----------------------------\nTop 3 closest results for king - man + woman:")
for word in allWords[:100]:   
    print(word.orth_)
    
# it got it! Queen!


----------------------------
Top 3 closest results for king - man + woman:
lady
girl
boy
men
guy
young
mother
she
women
gentleman
wife
he
teenager
husband
father
stranger
person
someone
female
who
victim
male
sex
lover
him
himself
herself
sexy
girlfriend
her
teenage
knows
dead
one
friend
when
gay
wants
mom
thing
girls
policeman
life
dude
cop
mans
boyfriend
soldier
never
pregnant
ladies
dad
drunk
knew
naked
being
ever
nothing
somebody
suspect
kind
his
fuck
say
teen
lad
face
whom
brother
prostitute
lives
thought
fucking
mind
told
that
way
alone
sexual
apparently
raped
looking
tells
housewife
horny
daughter
killed
lonely
tell
finds
blonde
boys
was
telling
innocent
wonder
once
death
couple
story


In [42]:
usnt = unicode('I shot an elephant')
#usnt = unicode(sentence)

from spacy.symbols import nsubj, VERB
# Finding a verb with a subject from below — good

verbs = []
for possible_verb in parser(usnt):
    if possible_verb.pos == VERB:
        for possible_subject in possible_verb.children:
            if possible_subject.dep == nsubj:
                verbs.append(possible_verb)
                break
                
print verbs

[shot]


In [44]:
usnt = unicode(sentence)
for word in parser(usnt):
    print(word.text, word.lemma, word.lemma_, word.tag, word.tag_, word.pos, word.pos_)

(u'A', 469, u'a', 426, u'DT', 87, u'DET')
(u'man', 698, u'man', 440, u'NN', 89, u'NOUN')
(u'and', 470, u'and', 424, u'CC', 86, u'CONJ')
(u'a', 469, u'a', 426, u'DT', 87, u'DET')
(u'woman', 1074, u'woman', 440, u'NN', 89, u'NOUN')
(u'standing', 1299, u'stand', 456, u'VBG', 97, u'VERB')
(u'in', 477, u'in', 432, u'IN', 83, u'ADP')
(u'the', 466, u'the', 426, u'DT', 87, u'DET')
(u'kitchen', 3633, u'kitchen', 440, u'NN', 89, u'NOUN')


In [48]:
# this one is cool:
usnt = unicode(sentence)
for np in parser(usnt).noun_chunks:
    print(np.text, np.root.text, np.root.dep_, np.root.head.text)


(u'A man', u'man', u'ROOT', u'man')
(u'a woman', u'woman', u'conj', u'man')
(u'the kitchen', u'kitchen', u'pobj', u'in')


In [52]:
def get_root(doc):
    tok = doc[0]
    while tok != tok.head:
        tok = tok.head

    return tok

def print_node(nlp, node, indent):
    print((' ' * indent) + nlp.vocab.strings[node.tag] + ' ' + str(node))
    for tok in node.children:
        if tok != node:
            print_node(nlp, tok, indent + 2)

def print_doc(nlp, doc):
    root = get_root(doc)
    print_node(nlp, root, indent=0)
    
print_doc(parser,parser(usnt))

NN man
  DT A
  CC and
  NN woman
    DT a
    VBG standing
      IN in
        NN kitchen
          DT the


In [113]:
from nltk.corpus import wordnet as wn
# take first item for word 'kitchen'
kitchen = wn.synsets('kitchen')[0]
paths = kitchen.hypernym_paths()
paths[0]


[Synset('entity.n.01'),
 Synset('physical_entity.n.01'),
 Synset('object.n.01'),
 Synset('whole.n.02'),
 Synset('artifact.n.01'),
 Synset('structure.n.01'),
 Synset('area.n.05'),
 Synset('room.n.01'),
 Synset('kitchen.n.01')]

In [234]:
import os, time,sys
from os.path import expanduser
home = expanduser("~")
pipe_name = home+'/text_transform'
pipein = open(pipe_name, 'r')
while True:
        line = pipein.readline()[:-1]
        print "got line:",line

got line: sadf
got line: ggasd


KeyboardInterrupt: 

'/Users/nesa/asdf'